## Deezer playlist dataset et recomandandation de morceaux avec word2vec

Dans ce mini projet nous allons mettre au point un réseau word2vec et nous en servir pour construire un outils de complétion de playlist (suggestion de morceaux). Les données sont hébergée sur le dépot suivant : http://github.com/comeetie/deezerplay.git. Pour en savoir plus sur word2vec et les données que nous allons utiliser vous pouvez lire les deux références sivantes :

- Efficient estimation of word representations in vector space, Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. (https://arxiv.org/abs/1301.3781)
- Word2vec applied to Recommendation: Hyperparameters Matter, H. Caselles-Dupré, F. Lesaint and J. Royo-Letelier. (https://arxiv.org/pdf/1804.04212.pdf)

Les éléments que vous devez réaliser sont mis en évidence en <span style="color:red">rouge</span> 


### Préparation des données
Les données sont sous la forme d'une liste de playlist. Chaque playlist est elle me une liste avec l'identifiant deezer du morçeau suivi de l'identifiant de l'artiste.

In [62]:
# chargement des données de playlist
import numpy as np
data = np.load("./music_2.npy",allow_pickle=True)
[len(data), np.mean([len(p) for p in data])]

[100000, 24.21338]

Le jeu de données sur lequel nous allons trvailler contient 100000 playlist qui sont composeer d'en moyenne 24.1 morceaux. Nous allons commencer par ne conserver que les identifiants de morceau. 

In [63]:
# separation des ids de morçeau et d'artist
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]
playlist_artist = [list(filter(lambda w: w.split("_")[0]==u"artist",playlist)) for playlist in data]

In [64]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

In [65]:
playlist_track[1]

['track_354917971',
 'track_354917981',
 'track_354917991',
 'track_354918001',
 'track_354918011',
 'track_405992352',
 'track_383552261',
 'track_397746652',
 'track_385499681',
 'track_402932992',
 'track_398786652',
 'track_357037431',
 'track_378113751',
 'track_354918011',
 'track_403520542',
 'track_393157072',
 'track_354165291',
 'track_357083271',
 'track_361171651',
 'track_145501426',
 'track_397141342',
 'track_145370202']

Le nombre de morceaux différents dans ce data-set est assez élevé avec plus de 300 000 morceaux.

### Création d'un dictionnnaire de morceau
Nous allons affecter a chaque morceau un entier qui nous servira d'identifiant unique et d'entrée pour notre réseau. Pour économiser un peu nos ressources nous allons travailler dans ce TP que sur les morceaux qui apparaissent dans au moins deux playlists.

In [66]:
# nombre d'occurence de chaque morceaux ?
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;

In [67]:
# Filtrage des morceaux peu fréquent pour gangner un peu de temps au vue de nos ressource en temps de calcul  
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 1, playlist)) for playlist in playlist_track]
# recupération des comptage
counts  =  np.array(list(track_counts.values()))
# trie
order = np.argsort(-counts)
# création de notre liste d'identifiant deezer
tracks_list_ordered = np.array(list(track_counts.keys()))[order]
# Taille de notre vocabulaire = nombre de morçeau conservés
Vt=np.where(counts[order]==1)[0][0]
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((tracks_list_ordered[i],i) for i in range(0, Vt))
# conversion des playlist en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

In [68]:
corpus_num_track[1]

[313,
 436,
 259,
 521,
 66,
 235,
 152,
 56,
 116,
 71,
 505,
 113,
 217,
 66,
 277,
 527,
 348,
 848,
 378,
 465,
 532,
 2016]

### Création des ensembles d'apprentissage de test et de validation

Pour apprendre les paramètre de notre méthode nous allons conserver les $l-1$ premiers morceaux de chaque playlist (avec $l$ la longueur de la playlist) pour l'apprentissage. Pour évaluer les performances de completion de notre méthode nous conservons pour chaque playlist les deux derniers morceaux. L'objectif sera de trouver le dernier a partir de l'avant dernier. 



In [69]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,20000)
index_val = np.setdiff1d(range(100000),index_tst)
# le debut de chaque playlist est conservé pour l'apprentissage
play_app  = [corpus_num_track[i][:(len(corpus_num_track[i])-1)] 
             for i in range(len(corpus_num_track)) if len(corpus_num_track[i])>1]
# les deux derniers élemnts pour le test et la validation
play_tst  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_tst if len(corpus_num_track[i])>3])
play_val  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_val if len(corpus_num_track[i])>3])[:10000]


In [70]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

### hyper-paramètres de word2vec :

La méthode word2vec fait intervennir un certains nombre d'hyper paramètres. Nous allons les définirs et leurs donner des première valeurs que nous affinerons par la suite:

In [71]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

### Création des tables de probabilité de tirage (lissée) et non lissée

Pour tirer les exmples négatif nous avons besoin des fréquence lissé de chaque morceau dans notre dataset. De même pour sous échantilloner les morceaux très fréquents nous avons besoin des fréquence brutes. Nous allons calculer ces deux vecteurs.

In [72]:
# recupération des comptage
counts = np.array(list(track_counts.values()),dtype='float')[order[:Vt]]
# normalisation
st =  counts/np.sum(counts)
# lissage
st_smooth = np.power(st,samp_coef)
st_smooth = st_smooth/np.sum(st_smooth)

### Construction du réseau word2vec

Un réseau word2vec prend en entrée deux entiers correspondant à deux morceaux, ceux-ci sont plonger dans un espace latent de dimension (vector_dim) grâce a une couche de type embedding (vous devrez utilisez la même couche pour projeter les deux morceaux). Une fois ces deux vecteurs extraits le réseau doit calculer leur produit scalaire normaliser appleler cosine distance : 

$$cos(\theta_{ij})=\frac{z_i.z_j}{||z_i||||z_j||}$$ 

Pour réaliser ce traitement vous utiliserez une couche "Dot" pour "dot product". Le modèle utilise ensuite une couche de type sigmoid pour produire la sortie. Cette sortie vaudra 0 lorsque les deux morceaux sont des morceaux tirés aléatoirement dans l'ensemble du jeu de donnée et 1 lorsqu'il aurront était extraits de la même playslist. <span style="color:red">A vous de créer le modèle keras Track2Vec correspondant à cette architecture.</span>

In [130]:
# entrée deux entier (couple de morceaux)
input_target = Input((1,), dtype='int32')
input_context = Input((1,), dtype='int32')

# a vous de compléter
embedded = Embedding(vector_dim, vector_dim, name="embedding-input")
embedded_target = embedded(input_target)
embedded_context = embedded(input_context)
dot_product = Dot(axes=1, name="dot")([embedded_target, embedded_context])
output = Dense(1, activation='sigmoid',name="classif")(dot_product)

# definition du modèle
Track2Vec = Model(inputs=[input_target, input_context], outputs=output)
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

In [131]:
Track2Vec.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_73 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_74 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding-inputtarget (Embeddi  (None, 1, 30)       900         ['input_73[0][0]',               
 ng)                                                              'input_74[0][0]']               
                                                                                                  
 dot (Dot)                      (None, 30, 30)       0           ['embedding-inputtarget[0][

### Création du générateur de données

Pour apprendre la couche de projection au coeur de notre modèle nous allons construire une générateur d'exemples positifs et négatifs de pair de morceaux proche ou aléatoires issues de nos données d'entrainement. La fonction suivante va permettre de générer de tels exemples a partir d'une playlist (seq) fournies en entrées. Cette fonction va tout d'abord construire tout les couples de morceau pouvant être extraient de la séquences s'ils se situent à moins de (windows) disance l'un de l'autres. Ces paires constitueront les paires positives. Les paires concernant deux morceaux très fréquents seront supprimer avec une probabilité qui dépendra de leur fréquences. Enfin un nombre d'exemple négatifs (correpondant neg_samples * nombre d'exemple positif) vont être tirés aléatoirement en utilisant la table de tirage (neg_sampling_table). 

In [132]:
# fonction générant les données associé a une séquence
# seq : séquence d'entrée
# neg_samples : nombre d'exemple négatif générés par example positif
# neg_sampling_table : probabilité de tirage des exemples négatif
# sub sampling_table : probabilité servant a sous échantilloner
# sub_t : paramètre de sous échantillonage
def word2vecSampling(seq,window,neg_samples,neg_sampling_table,sub_sampling_table,sub_t):
    # taille du vocabulaire
    V = len(neg_sampling_table)
    # créations des paires positives a partir de la séquence
    positives = skipgrams(sequence=seq, vocabulary_size=V, window_size=window,negative_samples=0)
    ppairs    = np.array(positives[0])
    # sous échantillonage
    if (ppairs.shape[0]>0):
        f = sub_sampling_table[ppairs[:,0]]
        subprob = ((f-sub_t)/f)-np.sqrt(sub_t/f)
        tokeep = (subprob<np.random.uniform(size=subprob.shape[0])) | (subprob<0)
        ppairs = ppairs[tokeep,:]
    nbneg     = ppairs.shape[0]*neg_samples
    # tirage des paires négatives
    if (nbneg > 0):
        negex     = np.random.choice(V, nbneg, p=neg_sampling_table)
        negexcontext = np.repeat(ppairs[:,0],neg_samples)
        npairs    = np.transpose(np.stack([negexcontext,negex]))
        pairs     = np.concatenate([ppairs,npairs],axis=0)
        labels    = np.concatenate([np.repeat(1,ppairs.shape[0]),np.repeat(0,nbneg)])
        perm      = np.random.permutation(len(labels))
        res = [pairs[perm,:],labels[perm]]
    else:
        res=[[],[]]
    return res

<span style="color:red">Utilisez cette fonction pour constuire un générateur "track_ns_generator" de données qui va générer des exemples positifs et négatifs à partir de "nbm" playlists tirées aléatoirement dans le jeu de données "corpus_num" fournis en entrée. </span>

In [146]:
# définition du générateur de couple de morceaux (y=0 <-> aléatoire, y=1 <-> proche dans une playlist)
import random
def track_ns_generator(corpus_num,nbm):
    #while True :
    # tirage de nbm playlist dans corpus_num
    x = []
    y = []
    for i in range(nbm) :
        playlist = random.choice(corpus_num)
        # création des données x et y 
        x1, y1 = word2vecSampling(playlist, window_width, neg_sample, st, st_smooth, sub_samp)
        #print(len(x1))
        #print(len(y1))
        for valx in x1 :
            x.append(valx)
        for valy in y1 :
            y.append(valy)
    return (x,y) 

track_ns_generator(play_app, min_batch_size)

([array([35364,    29]),
  array([14771,  1486]),
  array([14771,   130]),
  array([8457, 1025]),
  array([35364, 14771]),
  array([35364,  1001]),
  array([14771, 70849]),
  array([35364, 10020]),
  array([4235, 1931]),
  array([35364, 11148]),
  array([8457,   31]),
  array([35364,     2]),
  array([ 4235, 84589]),
  array([14771,  2221]),
  array([35364, 28815]),
  array([14771,  4235]),
  array([14771,  1546]),
  array([35364,   283]),
  array([ 8457, 35364]),
  array([ 8457, 10020]),
  array([4235,  350]),
  array([ 8457, 34682]),
  array([ 4235, 14771]),
  array([35364, 16010]),
  array([ 4235, 12733]),
  array([4235, 5278]),
  array([8457, 4235]),
  array([ 4235, 71098]),
  array([  8457, 109227]),
  array([14771,    62]),
  array([4235,   43]),
  array([4235, 4093]),
  array([8457, 3431]),
  array([4235, 8457]),
  array([14771, 68132]),
  array([35364, 21729]),
  array([4235, 8808]),
  array([35364, 55011]),
  array([ 4235, 35364]),
  array([35364, 34158]),
  array([14771, 3620

## Apprentissage 
Vous devriez maintenant être en mesure d'apprendre votre premier modèle avec le code suivant. Cela devrait durer entre 15 et 30 min.

In [147]:
hist=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=60)

ValueError: Failed to find data adapter that can handle input: (<class 'tuple'> containing values of types {'(<class \'list\'> containing values of types {"<class \'numpy.int64\'>"})', '(<class \'list\'> containing values of types {"<class \'numpy.ndarray\'>"})'}), <class 'NoneType'>

## Sauvegarde de l'espace latent
Nous pouvons une fois l'apprentissage effectué sauvegarder la position des morceaux dans l'espace latent avec le code suivant:

In [ ]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = Track2Vec.get_weights()[0]
with open('latent_positions.npy', 'wb') as f:
    np.save(f, vectors_tracks)

Et nous pouvons la recharger avec le code suivant :

In [ ]:
vectors_tracks=np.load("latent_positions.npy")

## Utilisation en complétion et évaluation
Nous pouvons maintenant nous servir de cet espace pour faire des suggestions. <span style="color:red">Construisez une fonction predict_batch qui prend en entrée un vecteur de numéro de morceaux (seeds), (s) un nombre de proposition a faire, les vecteurs des morceaux dans l'espace latent X et un kd-tree permettant d'accélérer les calculs de plus proche voisins. Pour faire ses propisitions cette fonctions retournera les indices des s plus proche voisins de chaque seeds. </span> Pour que ces predictions ne prennent pas trop de temps vous vous servirez d'un kd-tree (disponnible dans scikit learn) pour accélrer la recherche des plus proche voisins.

In [ ]:
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')

In [ ]:
def predict_batch(seeds,k,X,kdt):
    # TODO
    return 0

<span style="color:red">Utilisez cette fonction pour proposer des morceaux pour compléter les playlist du jeu de données de validation (les seeds correspondent à la première colone de play_val).</span>

In [ ]:
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)

<span style="color:red">Comparez ces suggestions avec la seconde colonne de play_val (les morceaux effectivement présents). Pour cela vous calculerez le hit@10 qui vaut 1 si le morceau effectivement présent dans la playlist fait partie des 10 propositions (ce score étant moyenné sur l'ensemble de validation) et le NDCG@10 (Normalized Discounted Cumulative Gain) qui prend en compte l'ordre des propositions. Ce second score vaut $1/log2(k+1)$ si la proposition k (k entre 1 et 10) est la proposition correcte et 0 si aucne proposition n'est correcte. Comme précedement vous calulerez le score moyen sur l'ensemble de validation. </span>


In [ ]:
NDGCatK

In [ ]:
HitatK

## Tunning des hyper paramètres

<span style="color:red">Vous pouvez maintenant essayer de faire varier les hyper paramètres pour améliorer vos performances. Attention au temps de calcul préparez une grille avec une dizaine de configurations différentes et évaluez chacune d'entre elles sur votre ensemblede validation.
Evaluez les performances finales de la meilleure configuration trouvée sur l'ensemble de test. N'oubliez pas de sauvergader vos résultats.</span>



## Bonus, un peu de musique

Le fichier TrackArtists contient des méta.données sur les morceaux et les artiste pour un sous ensemble des 300000 morceaux présent dans le dataset. Nous pouvons nous en servir pour recherchez le numéro d'un morceau a partir de son titre:

In [ ]:
import pandas as pd
tr_meta=pd.read_csv("./TracksArtists.csv")
joindf = pd.DataFrame({"track_id":tracks_list_ordered[:Vt],"index":range(Vt)})
meta = tr_meta.merge(joindf, left_on="track_id",right_on="track_id")
meta.set_index("index",inplace=True)
meta[["title","name","preview","track_id"]]

In [ ]:
def find_track(title):
    return meta.loc[meta["title"]==title,:].index[0]

tr=find_track("Hexagone")
tr

## Radio

L'api de deeezer permet de récupérer des informations sur les morceaux du dataset a partit de leur id deezer. Parmis ces infos lorsqu'elle est disponnible une url d'écoute d'un extrait gratuit est fournies.

In [ ]:
import urllib.request, json 
def gettrackinfo(number):
    track_url =  "https://api.deezer.com/track/{}".format(tracks_list_ordered[number].split("_")[1])
    with urllib.request.urlopen(track_url) as url:
        data = json.loads(url.read().decode())
    return data
track_apidata = gettrackinfo(find_track("Hexagone"))
track_apidata

Nous pouvons donc nous en servir pour écouter un extrait :

In [ ]:
from IPython.display import display, Audio, clear_output
display(Audio(track_apidata["preview"],autoplay=True))

<span style="color:red">Créez une fonction radio qui prend en entrée un numéro de morceau dans le dataset et lance une serie de nb_track morceaux en tirant aléatoirement dans le voisinage du morceau courant le morceau suivant a écouter. La taille du voisinage sera paramétrable et vous supprimerez des propositions les morceaux déjà écouté. Vous traiterez les exceptions si le morceau ne dispose pas d'extrait disponnible. Vous povez supprimer le morceau courant avec la fonction clear_display.</span>

In [ ]:
import time
def start_radio(seed,nb_candidates,duration,nbsteps=20):
    print(meta.loc[seed,"title"])
    display(Audio(meta.loc[seed,"preview"],autoplay=True))
    time.sleep(duration)
    clear_output()
    already_played = [seed]
    for i in range(nbsteps):
        try:
            # TODO
        except:
            print("track not found")
            pass
        clear_output()

In [ ]:
start_radio(find_track("Hexagone"),5,5,10)